In [ ]:
!pip install langchain

In [ ]:
!pip install -U langchain-community

In [30]:
%pip install --upgrade --quiet  langchain langchain-google-vertexai "langchain-google-community[featurestore]"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2.147.0 which is incompatible.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow-serving-api 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [41]:
from google.cloud import aiplatform
import google.cloud.bigquery as bq
import langchain
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from langchain.schema import format_document
from langchain_community.document_loaders import DirectoryLoader, TextLoader,BigQueryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore
from datetime import datetime
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore



PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-central1"

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAIBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAICommon has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [42]:
# Define our query
query = """
SELECT id,media_type,content,test_metadata 
FROM `nine-quality-test.Nine_Quality_Test.content_embeddings` ;
"""

# Load the data
loader = BigQueryLoader(
    query, metadata_columns=["id"], page_content_columns=["content","media_type","test_metadata"]
)


/var/tmp/ipykernel_710571/1683864506.py:8: LangChainDeprecationWarning: The class `BigQueryLoader` was deprecated in LangChain 0.0.32 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-google-community package and should be used instead. To use it run `pip install -U :class:`~langchain-google-community` and import as `from :class:`~langchain_google_community import BigQueryLoader``.
  loader = BigQueryLoader(


In [52]:
DATASET = "my_langchain_dataset"  # @param {type: "string"}
TABLE = "doc_and_vectors"  # @param {type: "string"}


embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)


store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
)

BigQuery table nine-quality-test.my_langchain_dataset.doc_and_vectors initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=nine-quality-test&ws=!1m5!1m4!4m3!1snine-quality-test!2smy_langchain_dataset!3sdoc_and_vectors


### source

https://cloud.google.com/blog/products/ai-machine-learning/rag-with-bigquery-and-langchain-in-cloud

In [45]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents = []
documents.extend(loader.load())
 
# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=5#,
   # separators=["\n\n"],
)
doc_splits = text_splitter.split_documents(documents)


# get current processing time to add it to metadata, datetime object containing current date and time
now = datetime.now()

# Add chunk number to metadata
chunk_idx=0
prev=doc_splits[0].metadata["id"]
for idx, split in enumerate(doc_splits):
    split.metadata["process_time"]=now
    if prev==split.metadata["id"]:
       split.metadata["chunk"] = chunk_idx      
    else:
        chunk_idx=0
        split.metadata["chunk"] = chunk_idx
        prev=split.metadata["id"]
    chunk_idx +=1
        

In [55]:

DATASET = "my_langchain_dataset"  # @param {type: "string"}
TABLE = "doc_and_vectors"  # @param {type: "string"}



embedding_model = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)
bq_store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
) 

_=bq_store.add_documents(doc_splits)

BigQuery table nine-quality-test.my_langchain_dataset.doc_and_vectors initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=nine-quality-test&ws=!1m5!1m4!4m3!1snine-quality-test!2smy_langchain_dataset!3sdoc_and_vectors


In [59]:
import google.cloud.bigquery as bq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore,BigQueryLoader
from datetime import datetime
import os
import logging
import json
    
def chunk_and_load():
    """
        Chunks the combination of page_content_columns from a given bigquery source string and generates
        text embeddings for them.

        Args:
            

        Returns:
             
        """
    
    project_id= os.environ.get("PROJECT_ID") 
    dataset= os.environ.get("DATASET")  
    table= os.environ.get("TABLE") 
    region= os.environ.get("REGION") 
    metadata_columns= str(os.environ.get("META_DATA_COLUMNS")).split(',') 
    page_content_columns= str(os.environ.get("PAGE_CONTENT_COLUMNS")).split(',') 
    source_query_str= os.environ.get("SOURCE_QUERY_STR") 
    separators= None if str(os.environ.get("SEPARATORS"))=="" else str(os.environ.get("SEPARATORS")).split(',') 
    chunk_size= os.environ.get("CHUNK_SIZE")
    chunk_overlap= 0 if os.environ.get("CHUNK_OVERLAP")=="" else os.environ.get("CHUNK_OVERLAP")
 

    project_id= 'nine-quality-test' 
    dataset= 'my_langchain_dataset'
    table= 'doc_and_vectors'
    region= 'us-central1'
    metadata_columns= "id".split(",")
    page_content_columns= "content,media_type,test_metadata".split(',') 
    source_query_str= """
    SELECT id,media_type,content,test_metadata 
    FROM `nine-quality-test.Nine_Quality_Test.content_embeddings` ;
    """
    separators= "\n\n"
    chunk_size= 25
    chunk_overlap= 5

     # Load the data
    loader = BigQueryLoader(
        source_query_str, metadata_columns=metadata_columns, page_content_columns=page_content_columns
    )

    documents = []
    documents.extend(loader.load())
    
    logging.info (f"Data Loaded from source - {source_query_str}")


    # Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
       separators=separators,
    )
    doc_splits = text_splitter.split_documents(documents)

    logging.info (f"Documents splitted - chunk_size {chunk_size}, chunk_overlap {chunk_overlap}")


    # get current processing time to add it to metadata, datetime object containing current date and time
    now = datetime.now()

    # Add chunk number to metadata
    chunk_idx=0
    prev=doc_splits[0].metadata["id"]
    rows_to_insert=[]
    for idx, split in enumerate(doc_splits):
        split.metadata["process_time"]=now
        if prev==split.metadata["id"]:
           split.metadata["chunk"] = chunk_idx      
        else:
            chunk_idx=0
            split.metadata["chunk"] = chunk_idx
            prev=split.metadata["id"]
        chunk_idx +=1

        rows_to_insert.append(
                           {"id": split.metadata["id"], 
                               "process_time":split.metadata["process_time"].isoformat(),
                               "content":  split.page_content,
                               "chunk": split.metadata["chunk"]
                              }
                                     )
    logging.info (f"Metadata added to chunks")

    from google.cloud import bigquery

    client = bigquery.Client()
    table_id = f"{project_id}.{dataset}.{table}"

 
    errors = client.insert_rows_json(table_id, rows_to_insert)
    if errors == []:
        print("New rows have been added.")
    else:
        print(f"Encountered errors: {errors}")
    
    
    
#     logging.info (f"Bigquery store info is set -  ProjectID {project_id}, Region {region}, Dataset {dataset}, Table {table}")


#     _=bq_store.add_documents(doc_splits)
    
#     logging.info (f"Chunks and embeddings added to the store")

    return 'done'

# if __name__ == "__main__":
#     project_id= os.environ.get("PROJECT_ID") 
#     dataset= os.environ.get("DATASET")  
#     table= os.environ.get("TABLE") 
#     region= os.environ.get("REGION") 
#     metadata_columns= str(os.environ.get("META_DATA_COLUMNS")).split(',') 
#     page_content_columns= str(os.environ.get("PAGE_CONTENT_COLUMNS")).split(',') 
#     source_query_str= os.environ.get("SOURCE_QUERY_STR") 
#     separators= None if str(os.environ.get("SEPARATORS"))=="" else str(os.environ.get("SEPARATORS")).split(',') 
#     chunk_size= os.environ.get("CHUNK_SIZE")
#     chunk_overlap= 0 if os.environ.get("CHUNK_OVERLAP")=="" else os.environ.get("CHUNK_OVERLAP")
 

#     project_id= 'nine-quality-test' 
#     dataset= 'my_langchain_dataset'
#     table= 'doc_and_vectors'
#     region= 'us-central1'
#     metadata_columns= "id".split(",")
#     page_content_columns= "content,media_type,test_metadata".split(',') 
#     source_query_str= """
#     SELECT id,media_type,content,test_metadata 
#     FROM `nine-quality-test.Nine_Quality_Test.content_embeddings` ;
#     """
#     separators= "\n\n"
#     chunk_size= 25
#     chunk_overlap= 5
    
    
#     message=chunk_and_load(project_id=project_id, dataset=dataset, table=table, region=region,\
#                         metadata_columns= metadata_columns, page_content_columns=page_content_columns, \
#                         source_query_str=source_query_str, separators=separators, chunk_size=chunk_size, \
#                        chunk_overlap=chunk_overlap)
#     return(message)
     

In [12]:
!pip install loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.4 MB/s eta 0:00:0000:01


In [17]:
import googleapiclient.discovery


def list_sub_directories(bucket_name, prefix):
    """Returns a list of sub-directories within the given bucket."""
    service = googleapiclient.discovery.build('storage', 'v1')

    req = service.objects().list(bucket=bucket_name, prefix=prefix, delimiter='/')
    
    res = req.execute()
    
    return res['prefixes']

# For the example (gs://abc/xyz), bucket_name is 'abc' and the prefix would be 'xyz/'
#print(list_sub_directories(bucket_name='raw_nine_files/sub_dir', prefix=''))

In [20]:
list(list_sub_directories(bucket_name=bucket, prefix=prefix))

['2023/', '2024/']

In [24]:
bucket='raw_nine_files'
prefix=''
l=[]
for main in list(list_sub_directories(bucket_name=bucket, prefix=prefix)):
    for dircty in list(list_sub_directories(bucket_name=bucket, prefix=main )):
       for sub_dircty in list(list_sub_directories(bucket_name=bucket, prefix=dircty )):
           print(sub_dircty)
        

2023/1/a/
2023/1/b/
2023/2/c/
2024/3/d/


In [ ]:
{
  "items": [
    {
      "bucket": "raw_nine_files",
      "contentType": "text/plain",
      "crc32c": "AAAAAA==",
      "etag": "CNvSrriv84gDEAE=",
      "eventBasedHold": false,
      "generation": "1727998336936283",
      "id": "raw_nine_files/2023//1727998336936283",
      "kind": "storage#object",
      "md5Hash": "1B2M2Y8AsgTpgAmY7PhCfg==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/2023%2F?generation=1727998336936283&alt=media",
      "metageneration": "1",
      "name": "2023/",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/2023%2F",
      "size": "0",
      "storageClass": "STANDARD",
      "temporaryHold": false,
      "timeCreated": "2024-10-03T23:32:16.992Z",
      "timeStorageClassUpdated": "2024-10-03T23:32:16.992Z",
      "updated": "2024-10-03T23:32:16.992Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "text/plain",
      "crc32c": "AAAAAA==",
      "etag": "CPOu/L2v84gDEAE=",
      "eventBasedHold": false,
      "generation": "1727998348695411",
      "id": "raw_nine_files/2023/1//1727998348695411",
      "kind": "storage#object",
      "md5Hash": "1B2M2Y8AsgTpgAmY7PhCfg==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/2023%2F1%2F?generation=1727998348695411&alt=media",
      "metageneration": "1",
      "name": "2023/1/",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/2023%2F1%2F",
      "size": "0",
      "storageClass": "STANDARD",
      "temporaryHold": false,
      "timeCreated": "2024-10-03T23:32:28.735Z",
      "timeStorageClassUpdated": "2024-10-03T23:32:28.735Z",
      "updated": "2024-10-03T23:32:28.735Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "text/plain",
      "crc32c": "AAAAAA==",
      "etag": "CNPa+MzB84gDEAE=",
      "eventBasedHold": false,
      "generation": "1728003211930963",
      "id": "raw_nine_files/2023/1/a//1728003211930963",
      "kind": "storage#object",
      "md5Hash": "1B2M2Y8AsgTpgAmY7PhCfg==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/2023%2F1%2Fa%2F?generation=1728003211930963&alt=media",
      "metageneration": "1",
      "name": "2023/1/a/",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/2023%2F1%2Fa%2F",
      "size": "0",
      "storageClass": "STANDARD",
      "temporaryHold": false,
      "timeCreated": "2024-10-04T00:53:31.970Z",
      "timeStorageClassUpdated": "2024-10-04T00:53:31.970Z",
      "updated": "2024-10-04T00:53:31.970Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "image/png",
      "crc32c": "mVwoYA==",
      "etag": "CI2v8/TB84gDEAE=",
      "generation": "1728003295729549",
      "id": "raw_nine_files/2023/1/a/7944pcnf.png/1728003295729549",
      "kind": "storage#object",
      "md5Hash": "DJbsdhDfC8KDJs38vknTng==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/2023%2F1%2Fa%2F7944pcnf.png?generation=1728003295729549&alt=media",
      "metageneration": "1",
      "name": "2023/1/a/7944pcnf.png",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/2023%2F1%2Fa%2F7944pcnf.png",
      "size": "352810",
      "storageClass": "STANDARD",
      "timeCreated": "2024-10-04T00:54:55.768Z",
      "timeStorageClassUpdated": "2024-10-04T00:54:55.768Z",
      "updated": "2024-10-04T00:54:55.768Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "image/png",
      "crc32c": "4Ydi3A==",
      "etag": "CIHTofzB84gDEAE=",
      "generation": "1728003311167873",
      "id": "raw_nine_files/2023/1/a/816f62dz.png/1728003311167873",
      "kind": "storage#object",
      "md5Hash": "Qhd9rXeQUdTFMs6lUHaWBg==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/2023%2F1%2Fa%2F816f62dz.png?generation=1728003311167873&alt=media",
      "metageneration": "1",
      "name": "2023/1/a/816f62dz.png",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/2023%2F1%2Fa%2F816f62dz.png",
      "size": "1013712",
      "storageClass": "STANDARD",
      "timeCreated": "2024-10-04T00:55:11.207Z",
      "timeStorageClassUpdated": "2024-10-04T00:55:11.207Z",
      "updated": "2024-10-04T00:55:11.207Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "text/plain",
      "crc32c": "AAAAAA==",
      "etag": "CND10dDB84gDEAE=",
      "eventBasedHold": false,
      "generation": "1728003219684048",
      "id": "raw_nine_files/2023/1/b//1728003219684048",
      "kind": "storage#object",
      "md5Hash": "1B2M2Y8AsgTpgAmY7PhCfg==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/2023%2F1%2Fb%2F?generation=1728003219684048&alt=media",
      "metageneration": "1",
      "name": "2023/1/b/",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/2023%2F1%2Fb%2F",
      "size": "0",
      "storageClass": "STANDARD",
      "temporaryHold": false,
      "timeCreated": "2024-10-04T00:53:39.724Z",
      "timeStorageClassUpdated": "2024-10-04T00:53:39.724Z",
      "updated": "2024-10-04T00:53:39.724Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "image/png",
      "crc32c": "mVwoYA==",
      "etag": "CJn2/IbC84gDEAE=",
      "generation": "1728003333634841",
      "id": "raw_nine_files/2023/1/b/7944pcnf.png/1728003333634841",
      "kind": "storage#object",
      "md5Hash": "DJbsdhDfC8KDJs38vknTng==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/2023%2F1%2Fb%2F7944pcnf.png?generation=1728003333634841&alt=media",
      "metageneration": "1",
      "name": "2023/1/b/7944pcnf.png",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/2023%2F1%2Fb%2F7944pcnf.png",
      "size": "352810",
      "storageClass": "STANDARD",
      "timeCreated": "2024-10-04T00:55:33.674Z",
      "timeStorageClassUpdated": "2024-10-04T00:55:33.674Z",
      "updated": "2024-10-04T00:55:33.674Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "text/plain",
      "crc32c": "AAAAAA==",
      "etag": "CPqCwKfB84gDEAE=",
      "eventBasedHold": false,
      "generation": "1728003133407610",
      "id": "raw_nine_files/2023/2//1728003133407610",
      "kind": "storage#object",
      "md5Hash": "1B2M2Y8AsgTpgAmY7PhCfg==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/2023%2F2%2F?generation=1728003133407610&alt=media",
      "metageneration": "1",
      "name": "2023/2/",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/2023%2F2%2F",
      "size": "0",
      "storageClass": "STANDARD",
      "temporaryHold": false,
      "timeCreated": "2024-10-04T00:52:13.448Z",
      "timeStorageClassUpdated": "2024-10-04T00:52:13.448Z",
      "updated": "2024-10-04T00:52:13.448Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "text/plain",
      "crc32c": "AAAAAA==",
      "etag": "CN+tqZ/C84gDEAE=",
      "eventBasedHold": false,
      "generation": "1728003384694495",
      "id": "raw_nine_files/2023/2/c//1728003384694495",
      "kind": "storage#object",
      "md5Hash": "1B2M2Y8AsgTpgAmY7PhCfg==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/2023%2F2%2Fc%2F?generation=1728003384694495&alt=media",
      "metageneration": "1",
      "name": "2023/2/c/",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/2023%2F2%2Fc%2F",
      "size": "0",
      "storageClass": "STANDARD",
      "temporaryHold": false,
      "timeCreated": "2024-10-04T00:56:24.733Z",
      "timeStorageClassUpdated": "2024-10-04T00:56:24.733Z",
      "updated": "2024-10-04T00:56:24.733Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "image/png",
      "crc32c": "39xg9A==",
      "etag": "CObQp6zC84gDEAE=",
      "generation": "1728003411929190",
      "id": "raw_nine_files/2023/2/c/screenshot_directrycontent.png/1728003411929190",
      "kind": "storage#object",
      "md5Hash": "YyYVJcx6AIHJWBpRS6AVFw==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/2023%2F2%2Fc%2Fscreenshot_directrycontent.png?generation=1728003411929190&alt=media",
      "metageneration": "1",
      "name": "2023/2/c/screenshot_directrycontent.png",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/2023%2F2%2Fc%2Fscreenshot_directrycontent.png",
      "size": "264106",
      "storageClass": "STANDARD",
      "timeCreated": "2024-10-04T00:56:51.967Z",
      "timeStorageClassUpdated": "2024-10-04T00:56:51.967Z",
      "updated": "2024-10-04T00:56:51.967Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "text/plain",
      "crc32c": "AAAAAA==",
      "etag": "CP76/5fB84gDEAE=",
      "eventBasedHold": false,
      "generation": "1728003100900734",
      "id": "raw_nine_files/2024//1728003100900734",
      "kind": "storage#object",
      "md5Hash": "1B2M2Y8AsgTpgAmY7PhCfg==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/2024%2F?generation=1728003100900734&alt=media",
      "metageneration": "1",
      "name": "2024/",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/2024%2F",
      "size": "0",
      "storageClass": "STANDARD",
      "temporaryHold": false,
      "timeCreated": "2024-10-04T00:51:40.943Z",
      "timeStorageClassUpdated": "2024-10-04T00:51:40.943Z",
      "updated": "2024-10-04T00:51:40.943Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "text/plain",
      "crc32c": "AAAAAA==",
      "etag": "CNLA5LXB84gDEAE=",
      "eventBasedHold": false,
      "generation": "1728003163365458",
      "id": "raw_nine_files/2024/3//1728003163365458",
      "kind": "storage#object",
      "md5Hash": "1B2M2Y8AsgTpgAmY7PhCfg==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/2024%2F3%2F?generation=1728003163365458&alt=media",
      "metageneration": "1",
      "name": "2024/3/",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/2024%2F3%2F",
      "size": "0",
      "storageClass": "STANDARD",
      "temporaryHold": false,
      "timeCreated": "2024-10-04T00:52:43.404Z",
      "timeStorageClassUpdated": "2024-10-04T00:52:43.404Z",
      "updated": "2024-10-04T00:52:43.404Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "text/plain",
      "crc32c": "AAAAAA==",
      "etag": "COez1bLC84gDEAE=",
      "eventBasedHold": false,
      "generation": "1728003425262055",
      "id": "raw_nine_files/2024/3/d//1728003425262055",
      "kind": "storage#object",
      "md5Hash": "1B2M2Y8AsgTpgAmY7PhCfg==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/2024%2F3%2Fd%2F?generation=1728003425262055&alt=media",
      "metageneration": "1",
      "name": "2024/3/d/",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/2024%2F3%2Fd%2F",
      "size": "0",
      "storageClass": "STANDARD",
      "temporaryHold": false,
      "timeCreated": "2024-10-04T00:57:05.301Z",
      "timeStorageClassUpdated": "2024-10-04T00:57:05.301Z",
      "updated": "2024-10-04T00:57:05.301Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "image/png",
      "crc32c": "39xg9A==",
      "etag": "CIa/lbvC84gDEAE=",
      "generation": "1728003443089286",
      "id": "raw_nine_files/2024/3/d/screenshot_directrycontent.png/1728003443089286",
      "kind": "storage#object",
      "md5Hash": "YyYVJcx6AIHJWBpRS6AVFw==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/2024%2F3%2Fd%2Fscreenshot_directrycontent.png?generation=1728003443089286&alt=media",
      "metageneration": "1",
      "name": "2024/3/d/screenshot_directrycontent.png",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/2024%2F3%2Fd%2Fscreenshot_directrycontent.png",
      "size": "264106",
      "storageClass": "STANDARD",
      "timeCreated": "2024-10-04T00:57:23.128Z",
      "timeStorageClassUpdated": "2024-10-04T00:57:23.128Z",
      "updated": "2024-10-04T00:57:23.128Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "text/vtt",
      "crc32c": "ruBjvQ==",
      "etag": "CL+q7pK9locDEAE=",
      "generation": "1720408514204991",
      "id": "raw_nine_files/60MI23_33_A.vtt/1720408514204991",
      "kind": "storage#object",
      "md5Hash": "6fDT10oHAURlgPiGYv4xtQ==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/60MI23_33_A.vtt?generation=1720408514204991&alt=media",
      "metageneration": "1",
      "name": "60MI23_33_A.vtt",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/60MI23_33_A.vtt",
      "size": "59463",
      "storageClass": "STANDARD",
      "timeCreated": "2024-07-08T03:15:14.277Z",
      "timeStorageClassUpdated": "2024-07-08T03:15:14.277Z",
      "updated": "2024-07-08T03:15:14.277Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "video/mp4",
      "crc32c": "hbkecw==",
      "etag": "CN3wiYu+locDEAE=",
      "generation": "1720408766314589",
      "id": "raw_nine_files/60MI23_33_A_HBB.mp4/1720408766314589",
      "kind": "storage#object",
      "md5Hash": "9YD4b5qfRaQnc14B1xGqiA==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/60MI23_33_A_HBB.mp4?generation=1720408766314589&alt=media",
      "metageneration": "1",
      "name": "60MI23_33_A_HBB.mp4",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/60MI23_33_A_HBB.mp4",
      "size": "1744302481",
      "storageClass": "STANDARD",
      "timeCreated": "2024-07-08T03:19:26.398Z",
      "timeStorageClassUpdated": "2024-07-08T03:19:26.398Z",
      "updated": "2024-07-08T03:19:26.398Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "video/mp4",
      "crc32c": "uk20GA==",
      "etag": "CLiO5P6SzIcDEAE=",
      "generation": "1722252623677240",
      "id": "raw_nine_files/60MI23_33_A_HBB_fixed.mp4/1722252623677240",
      "kind": "storage#object",
      "md5Hash": "DNNwjIclje86IcLi/b8pcw==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/60MI23_33_A_HBB_fixed.mp4?generation=1722252623677240&alt=media",
      "metageneration": "1",
      "name": "60MI23_33_A_HBB_fixed.mp4",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/60MI23_33_A_HBB_fixed.mp4",
      "size": "1743380331",
      "storageClass": "STANDARD",
      "timeCreated": "2024-07-29T11:30:23.799Z",
      "timeStorageClassUpdated": "2024-07-29T11:30:23.799Z",
      "updated": "2024-07-29T11:30:23.799Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "video/mp4",
      "crc32c": "e8YvXg==",
      "etag": "COPcr5raiYcDEAE=",
      "generation": "1719969637985891",
      "id": "raw_nine_files/60MI24_1_A_HBB.mp4/1719969637985891",
      "kind": "storage#object",
      "md5Hash": "GeMH1bSrAJi5vrBT9/+FWw==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/60MI24_1_A_HBB.mp4?generation=1719969637985891&alt=media",
      "metageneration": "1",
      "name": "60MI24_1_A_HBB.mp4",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/60MI24_1_A_HBB.mp4",
      "size": "1850846344",
      "storageClass": "STANDARD",
      "timeCreated": "2024-07-03T01:20:38.099Z",
      "timeStorageClassUpdated": "2024-07-03T01:20:38.099Z",
      "updated": "2024-07-03T01:20:38.099Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "video/mp4",
      "crc32c": "+e3auw==",
      "etag": "CMeo///ZiYcDEAE=",
      "generation": "1719969582666823",
      "id": "raw_nine_files/60MI24_6_A_HBB.mp4/1719969582666823",
      "kind": "storage#object",
      "md5Hash": "RkoXQoDitFU6uMxgcgZa2A==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/60MI24_6_A_HBB.mp4?generation=1719969582666823&alt=media",
      "metageneration": "1",
      "name": "60MI24_6_A_HBB.mp4",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/60MI24_6_A_HBB.mp4",
      "size": "1694178315",
      "storageClass": "STANDARD",
      "timeCreated": "2024-07-03T01:19:42.764Z",
      "timeStorageClassUpdated": "2024-07-03T01:19:42.764Z",
      "updated": "2024-07-03T01:19:42.764Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "image/png",
      "crc32c": "mVwoYA==",
      "etag": "CLnpgLiCsIgDEAE=",
      "generation": "1725684154119353",
      "id": "raw_nine_files/7944pcnf.png/1725684154119353",
      "kind": "storage#object",
      "md5Hash": "DJbsdhDfC8KDJs38vknTng==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/7944pcnf.png?generation=1725684154119353&alt=media",
      "metageneration": "1",
      "name": "7944pcnf.png",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/7944pcnf.png",
      "size": "352810",
      "storageClass": "STANDARD",
      "timeCreated": "2024-09-07T04:42:34.182Z",
      "timeStorageClassUpdated": "2024-09-07T04:42:34.182Z",
      "updated": "2024-09-07T04:42:34.182Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "image/png",
      "crc32c": "4Ydi3A==",
      "etag": "COyy2eue4ogDEAE=",
      "generation": "1727409765636460",
      "id": "raw_nine_files/816f62dz.png/1727409765636460",
      "kind": "storage#object",
      "md5Hash": "Qhd9rXeQUdTFMs6lUHaWBg==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/816f62dz.png?generation=1727409765636460&alt=media",
      "metageneration": "1",
      "name": "816f62dz.png",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/816f62dz.png",
      "size": "1013712",
      "storageClass": "STANDARD",
      "timeCreated": "2024-09-27T04:02:45.682Z",
      "timeStorageClassUpdated": "2024-09-27T04:02:45.682Z",
      "updated": "2024-09-27T04:02:45.682Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "text/vtt",
      "crc32c": "9QH4/g==",
      "etag": "CMehz8y+locDEAE=",
      "generation": "1720408903766215",
      "id": "raw_nine_files/MAAT2024_1_A.vtt/1720408903766215",
      "kind": "storage#object",
      "md5Hash": "9XM6K7ylDzgPbtXfQRqgEg==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/MAAT2024_1_A.vtt?generation=1720408903766215&alt=media",
      "metageneration": "1",
      "name": "MAAT2024_1_A.vtt",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/MAAT2024_1_A.vtt",
      "size": "125056",
      "storageClass": "STANDARD",
      "timeCreated": "2024-07-08T03:21:43.817Z",
      "timeStorageClassUpdated": "2024-07-08T03:21:43.817Z",
      "updated": "2024-07-08T03:21:43.817Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "video/mp4",
      "crc32c": "JGHveA==",
      "etag": "COG8t/W/locDEAE=",
      "generation": "1720409257795169",
      "id": "raw_nine_files/MAAT2024_1_A_HBB.mp4/1720409257795169",
      "kind": "storage#object",
      "md5Hash": "TU8sj6GWWmHXZAcLd5y6EQ==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/MAAT2024_1_A_HBB.mp4?generation=1720409257795169&alt=media",
      "metageneration": "1",
      "name": "MAAT2024_1_A_HBB.mp4",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/MAAT2024_1_A_HBB.mp4",
      "size": "3613937486",
      "storageClass": "STANDARD",
      "timeCreated": "2024-07-08T03:27:37.883Z",
      "timeStorageClassUpdated": "2024-07-08T03:27:37.883Z",
      "updated": "2024-07-08T03:27:37.883Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "text/vtt",
      "crc32c": "cGSAqw==",
      "etag": "COTjy8y+locDEAE=",
      "generation": "1720408903709156",
      "id": "raw_nine_files/MAAT2024_21_A.vtt/1720408903709156",
      "kind": "storage#object",
      "md5Hash": "iANJqndVNeynX09WNT3Uvg==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/MAAT2024_21_A.vtt?generation=1720408903709156&alt=media",
      "metageneration": "1",
      "name": "MAAT2024_21_A.vtt",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/MAAT2024_21_A.vtt",
      "size": "90296",
      "storageClass": "STANDARD",
      "timeCreated": "2024-07-08T03:21:43.751Z",
      "timeStorageClassUpdated": "2024-07-08T03:21:43.751Z",
      "updated": "2024-07-08T03:21:43.751Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "video/mp4",
      "crc32c": "yeP/0Q==",
      "etag": "COXB2M2/locDEAE=",
      "generation": "1720409174450405",
      "id": "raw_nine_files/MAAT2024_21_A_HBB.mp4/1720409174450405",
      "kind": "storage#object",
      "md5Hash": "dxbEeLad4dYjbO/hxIwHxg==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/MAAT2024_21_A_HBB.mp4?generation=1720409174450405&alt=media",
      "metageneration": "1",
      "name": "MAAT2024_21_A_HBB.mp4",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/MAAT2024_21_A_HBB.mp4",
      "size": "2748590138",
      "storageClass": "STANDARD",
      "timeCreated": "2024-07-08T03:26:14.535Z",
      "timeStorageClassUpdated": "2024-07-08T03:26:14.535Z",
      "updated": "2024-07-08T03:26:14.535Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "video/mp4",
      "crc32c": "vMtleQ==",
      "etag": "CMqKxIHeiYcDEAE=",
      "generation": "1719970659632458",
      "id": "raw_nine_files/MAAT2024_36_A_HBB.mp4/1719970659632458",
      "kind": "storage#object",
      "md5Hash": "SfqWK0/55DySEQ3Iv1xhKg==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/MAAT2024_36_A_HBB.mp4?generation=1719970659632458&alt=media",
      "metageneration": "1",
      "name": "MAAT2024_36_A_HBB.mp4",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/MAAT2024_36_A_HBB.mp4",
      "size": "2508037349",
      "storageClass": "STANDARD",
      "timeCreated": "2024-07-03T01:37:39.742Z",
      "timeStorageClassUpdated": "2024-07-03T01:37:39.742Z",
      "updated": "2024-07-03T01:37:39.742Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "text/vtt",
      "crc32c": "cQOFfw==",
      "etag": "CNOo+5O9locDEAE=",
      "generation": "1720408516514899",
      "id": "raw_nine_files/NNNTW24_21_A.vtt/1720408516514899",
      "kind": "storage#object",
      "md5Hash": "Nzb4pOnptkW+4dF7duru9A==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/NNNTW24_21_A.vtt?generation=1720408516514899&alt=media",
      "metageneration": "1",
      "name": "NNNTW24_21_A.vtt",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/NNNTW24_21_A.vtt",
      "size": "116881",
      "storageClass": "STANDARD",
      "timeCreated": "2024-07-08T03:15:16.557Z",
      "timeStorageClassUpdated": "2024-07-08T03:15:16.557Z",
      "updated": "2024-07-08T03:15:16.557Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "text/vtt",
      "crc32c": "klCLCA==",
      "etag": "CK3txee9locDEAE=",
      "generation": "1720408691799725",
      "id": "raw_nine_files/NNNTW724_21_A.vtt/1720408691799725",
      "kind": "storage#object",
      "md5Hash": "90xzoKT3BiQwo6VB0KnNog==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/NNNTW724_21_A.vtt?generation=1720408691799725&alt=media",
      "metageneration": "1",
      "name": "NNNTW724_21_A.vtt",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/NNNTW724_21_A.vtt",
      "size": "121538",
      "storageClass": "STANDARD",
      "timeCreated": "2024-07-08T03:18:11.867Z",
      "timeStorageClassUpdated": "2024-07-08T03:18:11.867Z",
      "updated": "2024-07-08T03:18:11.867Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "video/mp4",
      "crc32c": "L8Xl2w==",
      "etag": "CImIl+q9locDEAE=",
      "generation": "1720408697324553",
      "id": "raw_nine_files/SYD-NINE_NNNTW24_21_A.mp4/1720408697324553",
      "kind": "storage#object",
      "md5Hash": "3YZ1foJ5hXdkPUdUxBok7Q==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/SYD-NINE_NNNTW24_21_A.mp4?generation=1720408697324553&alt=media",
      "metageneration": "1",
      "name": "SYD-NINE_NNNTW24_21_A.mp4",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/SYD-NINE_NNNTW24_21_A.mp4",
      "size": "1094890707",
      "storageClass": "STANDARD",
      "timeCreated": "2024-07-08T03:18:17.402Z",
      "timeStorageClassUpdated": "2024-07-08T03:18:17.402Z",
      "updated": "2024-07-08T03:18:17.402Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "video/mp4",
      "crc32c": "e2zh1Q==",
      "etag": "CJ3swea9locDEAE=",
      "generation": "1720408689636893",
      "id": "raw_nine_files/SYD-NINE_NNNTW724_21_A.mp4/1720408689636893",
      "kind": "storage#object",
      "md5Hash": "wf2OgwkOiRMTr9Pk5lD9gQ==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/SYD-NINE_NNNTW724_21_A.mp4?generation=1720408689636893&alt=media",
      "metageneration": "1",
      "name": "SYD-NINE_NNNTW724_21_A.mp4",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/SYD-NINE_NNNTW724_21_A.mp4",
      "size": "1109220796",
      "storageClass": "STANDARD",
      "timeCreated": "2024-07-08T03:18:09.706Z",
      "timeStorageClassUpdated": "2024-07-08T03:18:09.706Z",
      "updated": "2024-07-08T03:18:09.706Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "video/mp4",
      "crc32c": "G6uzEA==",
      "etag": "CLmfoInilocDEAE=",
      "generation": "1720418426163129",
      "id": "raw_nine_files/episode-18-brs-full-story.mp4/1720418426163129",
      "kind": "storage#object",
      "md5Hash": "KPfFkaFpjt/fec6wl3wZaw==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/episode-18-brs-full-story.mp4?generation=1720418426163129&alt=media",
      "metageneration": "1",
      "name": "episode-18-brs-full-story.mp4",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/episode-18-brs-full-story.mp4",
      "size": "7183155286",
      "storageClass": "STANDARD",
      "timeCreated": "2024-07-08T06:00:26.231Z",
      "timeStorageClassUpdated": "2024-07-08T06:00:26.231Z",
      "updated": "2024-07-08T06:00:26.231Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "text/vtt",
      "crc32c": "DWV1lw==",
      "etag": "COaPt43blocDEAE=",
      "generation": "1720416555878374",
      "id": "raw_nine_files/episode-18-brs-full-story.vtt/1720416555878374",
      "kind": "storage#object",
      "md5Hash": "k7bNeIqqbNBbyyjSyBMCPg==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/episode-18-brs-full-story.vtt?generation=1720416555878374&alt=media",
      "metageneration": "1",
      "name": "episode-18-brs-full-story.vtt",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/episode-18-brs-full-story.vtt",
      "size": "78402",
      "storageClass": "STANDARD",
      "timeCreated": "2024-07-08T05:29:15.927Z",
      "timeStorageClassUpdated": "2024-07-08T05:29:15.927Z",
      "updated": "2024-07-08T05:29:15.927Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "application/json; charset=UTF-8",
      "crc32c": "Bz2luQ==",
      "etag": "CLD2ruq64ogDEAE=",
      "generation": "1727417279036208",
      "id": "raw_nine_files/output.json/1727417279036208",
      "kind": "storage#object",
      "md5Hash": "PUMfOiBfyaU2Ioe8ev3BKg==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/output.json?generation=1727417279036208&alt=media",
      "metageneration": "1",
      "name": "output.json",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/output.json",
      "size": "101",
      "storageClass": "STANDARD",
      "timeCreated": "2024-09-27T06:07:59.146Z",
      "timeStorageClassUpdated": "2024-09-27T06:07:59.146Z",
      "updated": "2024-09-27T06:07:59.146Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "image/jpeg",
      "crc32c": "5MKdfA==",
      "etag": "CNjEo+ye4ogDEAE=",
      "generation": "1727409766851160",
      "id": "raw_nine_files/sport2.jpg/1727409766851160",
      "kind": "storage#object",
      "md5Hash": "y49z3MmysaKWotJA+6z5OA==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/sport2.jpg?generation=1727409766851160&alt=media",
      "metageneration": "1",
      "name": "sport2.jpg",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/sport2.jpg",
      "size": "2793769",
      "storageClass": "STANDARD",
      "timeCreated": "2024-09-27T04:02:46.904Z",
      "timeStorageClassUpdated": "2024-09-27T04:02:46.904Z",
      "updated": "2024-09-27T04:02:46.904Z"
    },
    {
      "bucket": "raw_nine_files",
      "contentType": "image/png",
      "crc32c": "PQC7HA==",
      "etag": "COK53uue4ogDEAE=",
      "generation": "1727409765719266",
      "id": "raw_nine_files/v5c0kpyo.png/1727409765719266",
      "kind": "storage#object",
      "md5Hash": "akhZzkC8i8eqX0ApAZO69w==",
      "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/raw_nine_files/o/v5c0kpyo.png?generation=1727409765719266&alt=media",
      "metageneration": "1",
      "name": "v5c0kpyo.png",
      "selfLink": "https://www.googleapis.com/storage/v1/b/raw_nine_files/o/v5c0kpyo.png",
      "size": "1524329",
      "storageClass": "STANDARD",
      "timeCreated": "2024-09-27T04:02:45.771Z",
      "timeStorageClassUpdated": "2024-09-27T04:02:45.771Z",
      "updated": "2024-09-27T04:02:45.771Z"
    }
  ],
  "kind": "storage#objects"
}

In [97]:
def list_gcs_objects(bucket_name):
    """List all objects in a Google Cloud Storage bucket and print their metadata."""
    # Initialize a storage client
    storage_client = storage.Client()

    # Get the bucket
    bucket = storage_client.get_bucket(bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs()

    print(f"Objects in bucket '{bucket_name}':")
    for blob in blobs:
        print(f"Name: {blob.name}")
        print(f"Size: {blob.size} bytes")
        print(f"Content Type: {blob.content_type}")
        print(f"Media Link: {blob.public_url}")  # Get public URL if the object is publicly accessible
        print(f"Created: {blob.time_created}")
        print(f"Updated: {blob.updated}")
        print(f"Updated: {blob.self_link}")
        
        print("------")
    

In [98]:
# Call the function with your bucket name
bucket_name = 'raw_nine_files'
list_gcs_objects(bucket_name)

Objects in bucket 'raw_nine_files':
Name: 2023/
Size: 0 bytes
Content Type: text/plain
Media Link: https://storage.googleapis.com/raw_nine_files/2023/
Created: 2024-10-03 23:32:16.992000+00:00
Updated: 2024-10-03 23:32:16.992000+00:00
Updated: https://www.googleapis.com/storage/v1/b/raw_nine_files/o/2023%2F
------
Name: 2023/1/
Size: 0 bytes
Content Type: text/plain
Media Link: https://storage.googleapis.com/raw_nine_files/2023/1/
Created: 2024-10-03 23:32:28.735000+00:00
Updated: 2024-10-03 23:32:28.735000+00:00
Updated: https://www.googleapis.com/storage/v1/b/raw_nine_files/o/2023%2F1%2F
------
Name: 2023/1/a/
Size: 0 bytes
Content Type: text/plain
Media Link: https://storage.googleapis.com/raw_nine_files/2023/1/a/
Created: 2024-10-04 00:53:31.970000+00:00
Updated: 2024-10-04 00:53:31.970000+00:00
Updated: https://www.googleapis.com/storage/v1/b/raw_nine_files/o/2023%2F1%2Fa%2F
------
Name: 2023/1/a/7944pcnf.png
Size: 352810 bytes
Content Type: image/png
Media Link: https://storage.g

In [73]:
from google.cloud import storage

def list_gcs_objects(bucket_name):
    """List all objects in a Google Cloud Storage bucket."""
    # Initialize a client for interacting with GCS
    storage_client = storage.Client()

    # Get the bucket by name
    bucket = storage_client.get_bucket(bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs()

    print(f"Objects in bucket {bucket_name}:")
    for blob in blobs:
        blob = bucket.get_blob(blob.name)

        print(blob.mediaLink)

# Call the function with your bucket name
bucket_name = 'raw_nine_files'
list_gcs_objects(bucket_name)

Objects in bucket raw_nine_files:


AttributeError: 'Blob' object has no attribute 'mediaLink'

In [95]:
import google.cloud.bigquery as bq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore,BigQueryLoader
from datetime import datetime
import os
import logging
import json
    
def chunk_and_load(project_id: str= None, dataset: str= None, table: str= None, region: str =None,\
                        metadata_columns: list[str]=None, page_content_columns: list[str]= None, \
                        source_query_str: str= None, separators:  list[str]=None, chunk_size: int=None, \
                       chunk_overlap: int=0):
    """
        Chunks the combination of page_content_columns from a given bigquery source string and generates
        text embeddings for them.

        Args:
            

        Returns:
             
        """


     # Load the data
    loader = BigQueryLoader(
        source_query_str, metadata_columns=metadata_columns, page_content_columns=page_content_columns
    )

    documents = []
    documents.extend(loader.load())
    
    logging.info (f"Data Loaded from source - {source_query_str}")


    # Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
       separators=separators,
    )
    doc_splits = text_splitter.split_documents(documents)
    
   
    logging.info (f"Documents splitted - chunk_size {chunk_size}, chunk_overlap {chunk_overlap}")


    # get current processing time to add it to metadata, datetime object containing current date and time
    now = datetime.now()

    # Add chunk number to metadata
    chunk_idx=0
    prev=doc_splits[0].metadata["id"]
    rows_to_insert=[]
    for idx, split in enumerate(doc_splits):
        split.metadata["process_time"]=now
        if prev==split.metadata["id"]:
           split.metadata["chunk"] = chunk_idx      
        else:
            chunk_idx=0
            split.metadata["chunk"] = chunk_idx
            prev=split.metadata["id"]
        chunk_idx +=1

        rows_to_insert.append(
                           {"id": split.metadata["id"], 
                               "process_time":split.metadata["process_time"].isoformat(),
                               "content":  split.page_content,
                               "chunk": split.metadata["chunk"]
                              }
                                     )
    logging.info (f"Metadata added to chunks")

    from google.cloud import bigquery

    client = bigquery.Client()
    table_id = f"{project_id}.{dataset}.{table}"

 
    errors = client.insert_rows_json(table_id, rows_to_insert)
    if errors == []:
        print("New rows have been added.")
    else:
        print(f"Encountered errors: {errors}")
    
    
    
#     logging.info (f"Bigquery store info is set -  ProjectID {project_id}, Region {region}, Dataset {dataset}, Table {table}")


#     _=bq_store.add_documents(doc_splits)
    
#     logging.info (f"Chunks and embeddings added to the store")

    return 'done'

if __name__ == "__main__":
    project_id= os.environ.get("PROJECT_ID") 
    dataset= os.environ.get("DATASET")  
    table= os.environ.get("TABLE") 
    region= os.environ.get("REGION") 
    metadata_columns= str(os.environ.get("META_DATA_COLUMNS")).split(',') 
    page_content_columns= str(os.environ.get("PAGE_CONTENT_COLUMNS")).split(',') 
    source_query_str= os.environ.get("SOURCE_QUERY_STR") 
    separators= None if str(os.environ.get("SEPARATORS"))=="" else str(os.environ.get("SEPARATORS")).split(',') 
    chunk_size= 1000 if str(os.environ.get("CHUNK_SIZE"))=="None" else int(str(os.environ.get("CHUNK_SIZE")))  
    chunk_overlap= 0 if str(os.environ.get("CHUNK_OVERLAP"))=="None" else int(str(os.environ.get("CHUNK_OVERLAP")))
 
 

    project_id= 'nine-quality-test' 
    dataset= 'my_langchain_dataset'
    table= 'doc_and_vectors'
    region= 'us-central1'
    metadata_columns= "id".split(",")
    page_content_columns= "content,media_type,test_metadata".split(',') 
    source_query_str= """
    SELECT id,media_type,content,test_metadata 
    FROM `nine-quality-test.Nine_Quality_Test.content_embeddings` ;
    """
    separators= "\\n\\n"
    chunk_size= 25
    chunk_overlap= 5
    
    
    message=chunk_and_load(project_id=project_id, dataset=dataset, table=table, region=region,\
                        metadata_columns= metadata_columns, page_content_columns=page_content_columns, \
                        source_query_str=source_query_str, separators=separators, chunk_size=chunk_size, \
                       chunk_overlap=chunk_overlap)
    print(message)

New rows have been added.
done


In [96]:
str(os.environ.get("CHUNK_SIZE"))

'None'

In [115]:
from google.cloud import storage
import tempfile, shutil
import time
from datetime import datetime
import json



def create_image_request_file( dest_bucket_name: str= None, source_bucket_name: str= None,
                          request_file_prefix: str= None, request_file_folder: str= None, mime_types: str= None,
                          prompt_text: str="", temperature: float= None,
                          max_output_tokens: int=2048, top_p: float= None, top_k : float= None):
    """create batch request file and store it in gcs
   
        Args:
            str dest_bucket_name: name of the destination gcs bucket to write files to
            str dest_folder_name: name of the destination folder name to write files to
            str file_pre_fix: prefix of the destination file name
            list requests: list of requests

        Returns:
            str path:  path to the gcs request file object
          
    """
        
      # Create temporary file to write summaries to
    request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=False) 

    # Initialize a client for interacting with GCS
    storage_client = storage.Client()
    # Get the bucket by name
    bucket = storage_client.get_bucket(source_bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs()

    with open(request_file.name, "a") as rf :
         for blob in blobs:
            if blob.content_type in mime_types:
                 gcsuri= "gs://"+source_bucket_name+"/"+blob.name
                 mimeType=blob.content_type

                 request_list=[
                       json.dumps(
                              {
                                "request": 
                                     {
                                      "contents": 
                                          {"parts": [{ "fileData": 
                                                         {"fileUri": gcsuri, "mimeType": mimeType}
                                                      }, 
                                                      {"text": prompt_text
                                                      }
                                                    ]
                                                      , 
                                            "role": "user"
                                          }
                                          , 
                                          "generation_config": 
                                               {"max_output_tokens": max_output_tokens, 
                                                "temperature":temperature, 
                                                 "top_k": top_k, 
                                                 "top_p": top_p
                                                }
                                          , 
                                          "safety_settings": 
                                           [
                                            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", 
                                             "threshold": "BLOCK_NONE"
                                             }
                                           , 
                                           {"category": "HARM_CATEGORY_HATE_SPEECH", 
                                           "threshold": "BLOCK_NONE"
                                           },
                                           {"category": "HARM_CATEGORY_HARASSMENT", 
                                           "threshold": "BLOCK_NONE"
                                           }, 
                                           {"category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                                            "threshold": "BLOCK_NONE"
                                            }


                                           ]
                                     }
                              }
                       )  +"\n"
                 ]

                 rf.writelines(request_list)

    temp=request_file
    client = storage.Client()
    now=datetime.strptime(str(datetime.now()),
                               '%Y-%m-%d %H:%M:%S.%f')
    # Extract name to the temp file
    temp_file = "".join([str(temp.name)])
    # Uploading the temp image file to the bucket
    dest_filename = f"{request_file_folder}/"+request_file_prefix+datetime.strftime(now, '%Y%m%d%H%M%S')+".json" 
    dest_bucket = client.get_bucket(dest_bucket_name)
    dest_blob = dest_bucket.blob(dest_filename)
    dest_blob.upload_from_filename(temp_file)

    return dest_filename

if 1==1:
 

    dest_bucket_name ='artifacts-nine-quality-test-embeddings' #request_args['destination_bucket']
    source_bucket_name ='raw_nine_files'# request_args['source_bucket']
    request_file_prefix = "image_request"#request_args['request_file_prefix']
    request_file_folder =  "image_batch_request_fldr"#request_args['request_file_folder']
    prompt_text= "You are an assistant tasked with summarizing images for retrieval. \\ These summaries will be embedded and used to retrieve the raw image. \\ Give a concise summary of the image that is well optimized for retrieval.\\ If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\\ If you are not sure about any info, please do not make it up. Do not add any extra text to the output."#request_args['prompt_text']
    media_types= ['image/jpeg','image/png']#list(request_args['media_type'])
    request_content= "image"#request_args['request_content']
 
    temperature=None
    max_output_tokens=2048
    top_p=None
    top_k=None
  
    if  request_content=='image':
      dest_filename=create_image_request_file(dest_bucket_name=dest_bucket_name,source_bucket_name=source_bucket_name,
                                      request_file_prefix=request_file_prefix,request_file_folder=request_file_folder,
                                      mime_types=media_types, prompt_text=prompt_text,temperature=temperature,
                                     max_output_tokens=max_output_tokens,top_p=top_p,top_k=top_k
                                      )  



In [112]:
dest_filename

'image_batch_request_fldr/image_request20241005084038.json'

In [118]:
from google.cloud import storage
import tempfile, shutil
import time
from datetime import datetime
import json


def create_image_request_file( dest_bucket_name: str= None, source_bucket_name: str= None,
                          request_file_prefix: str= None, request_file_folder: str= None, mime_types: str= None,
                          prompt_text: str="", temperature: float= None,
                          max_output_tokens: int=2048, top_p: float= None, top_k : float= None):
    """create batch request file and store it in gcs
   
        Args:
            str dest_bucket_name: name of the destination gcs bucket to write files to
            str dest_folder_name: name of the destination folder name to write files to
            str file_pre_fix: prefix of the destination file name
            list requests: list of requests

        Returns:
            str path:  path to the gcs request file object
          
    """
        
      # Create temporary file to write summaries to
    request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=False) 

    # Initialize a client for interacting with GCS
    storage_client = storage.Client()
    # Get the bucket by name
    bucket = storage_client.get_bucket(source_bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs()

    with open(request_file.name, "a") as rf :
         for blob in blobs:
            if blob.content_type in mime_types:
                 gcsuri= "gs://"+source_bucket_name+"/"+blob.name
                 mimeType=blob.content_type

                 request_list=[
                       json.dumps(
                              {
                                "request": 
                                     {
                                      "contents": 
                                          {"parts": [{ "fileData": 
                                                         {"fileUri": gcsuri, "mimeType": mimeType}
                                                      }, 
                                                      {"text": prompt_text
                                                      }
                                                    ]
                                                      , 
                                            "role": "user"
                                          }
                                          , 
                                          "generation_config": 
                                               {"max_output_tokens": max_output_tokens, 
                                                "temperature":temperature, 
                                                 "top_k": top_k, 
                                                 "top_p": top_p
                                                }
                                          , 
                                          "safety_settings": 
                                           [
                                            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", 
                                             "threshold": "BLOCK_NONE"
                                             }
                                           , 
                                           {"category": "HARM_CATEGORY_HATE_SPEECH", 
                                           "threshold": "BLOCK_NONE"
                                           },
                                           {"category": "HARM_CATEGORY_HARASSMENT", 
                                           "threshold": "BLOCK_NONE"
                                           }, 
                                           {"category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                                            "threshold": "BLOCK_NONE"
                                            }


                                           ]
                                     }
                              }
                       )  +"\n"
                 ]

                 rf.writelines(request_list)

    temp=request_file
    client = storage.Client()
    now=datetime.strptime(str(datetime.now()),
                               '%Y-%m-%d %H:%M:%S.%f')
    # Extract name to the temp file
    temp_file = "".join([str(temp.name)])
    # Uploading the temp image file to the bucket
    dest_filename = f"{request_file_folder}/"+request_file_prefix+datetime.strftime(now, '%Y%m%d%H%M%S')+".json" 
    dest_bucket = client.get_bucket(dest_bucket_name)
    dest_blob = dest_bucket.blob(dest_filename)
    dest_blob.upload_from_filename(temp_file)

    return dest_filename

if 1==1:
    """HTTP Cloud Function.
    Args:
        request (flask.Request): The request object.
        <https://flask.palletsprojects.com/en/1.1.x/api/#incoming-request-data>
    Returns:
        The response text, or any set of values that can be turned into a
        Response object using `make_response`
        <https://flask.palletsprojects.com/en/1.1.x/api/#flask.make_response>.
    """
 

    # dest_bucket_name =request_args['destination_bucket']
    # source_bucket_name =request_args['source_bucket']
    # request_file_prefix =request_args['request_file_prefix']
    # request_file_folder =request_args['request_file_folder']
    # prompt_text= request_args['prompt_text']
    # media_types= list(request_args['media_type'])
    # request_content= request_args['request_content']

    # if request_args and 'temperature' in request_args:
    #     temperature= request_args['temperature']
    # else:
    #   temperature=0.5

    # if request_args and 'max_output_tokens' in request_args:
    #    max_output_tokens= request_args['max_output_tokens'] 
    # else:
    #      max_output_tokens=2048

    # if request_args and 'top_p' in request_args:
    #     top_p= request_args['top_p']
    # else:
    #      top_p=0.5

    # if request_args and 'top_k' in request_args:
    #     top_k= request_args['top_k']
    # else:
    #      top_k=50
  
  
    dest_bucket_name ='artifacts-nine-quality-test-embeddings' #request_args['destination_bucket']
    source_bucket_name ='raw_nine_files'# request_args['source_bucket']
    request_file_prefix = "image_request"#request_args['request_file_prefix']
    request_file_folder =  "image_batch_request_fldr"#request_args['request_file_folder']
    prompt_text= "You are an assistant tasked with summarizing images for retrieval. \\ These summaries will be embedded and used to retrieve the raw image. \\ Give a concise summary of the image that is well optimized for retrieval.\\ If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\\ If you are not sure about any info, please do not make it up. Do not add any extra text to the output."#request_args['prompt_text']
    media_types= ['image/jpeg','image/png']#list(request_args['media_type'])
    request_content= "image"#request_args['request_content']
 
    temperature=0.5
    max_output_tokens=2048
    top_p=50
    top_k=0.5

    if  request_content=='image':
      dest_filename=create_image_request_file(dest_bucket_name=dest_bucket_name,source_bucket_name=source_bucket_name,
                                      request_file_prefix=request_file_prefix,request_file_folder=request_file_folder,
                                      mime_types=media_types, prompt_text=prompt_text,temperature=temperature,
                                     max_output_tokens=max_output_tokens,top_p=top_p,top_k=top_k
                                      )  

 

In [120]:
S="['image/jpeg','image/png']"

list(S)

['[',
 "'",
 'i',
 'm',
 'a',
 'g',
 'e',
 '/',
 'j',
 'p',
 'e',
 'g',
 "'",
 ',',
 "'",
 'i',
 'm',
 'a',
 'g',
 'e',
 '/',
 'p',
 'n',
 'g',
 "'",
 ']']

In [ ]:
import functions_framework
from google.cloud import storage
import tempfile, shutil
import time
from datetime import datetime
import json


def create_image_request_file( dest_bucket_name: str= None, source_bucket_name: str= None,
                          request_file_prefix: str= None, request_file_folder: str= None, mime_types: list[str]= None,
                          prompt_text: str="", temperature: float= None,
                          max_output_tokens: int=2048, top_p: float= None, top_k : float= None):
    """create batch request file and store it in gcs
   
        Args:
            str dest_bucket_name: name of the destination gcs bucket to write files to
            str dest_folder_name: name of the destination folder name to write files to
            str file_pre_fix: prefix of the destination file name
            list requests: list of requests

        Returns:
            str path:  path to the gcs request file object
          
    """
        
      # Create temporary file to write summaries to
    request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=False) 

    # Initialize a client for interacting with GCS
    storage_client = storage.Client()
    # Get the bucket by name
    bucket = storage_client.get_bucket(source_bucket_name)

    # List all objects in the bucket
    blobs = bucket.list_blobs()
    
    with open(request_file.name, "a") as rf :
         for blob in blobs:
            if blob.content_type in mime_types:
                 gcsuri= "gs://"+source_bucket_name+"/"+blob.name
                 mimeType=blob.content_type

                 request_list=[
                       json.dumps(
                              {
                                "request": 
                                     {
                                      "contents": 
                                          {"parts": [{ "fileData": 
                                                         {"fileUri": gcsuri, "mimeType": mimeType}
                                                      }, 
                                                      {"text": prompt_text
                                                      }
                                                    ]
                                                      , 
                                            "role": "user"
                                          }
                                          , 
                                          "generation_config": 
                                               {"max_output_tokens": max_output_tokens, 
                                                "temperature":temperature, 
                                                 "top_k": top_k, 
                                                 "top_p": top_p
                                                }
                                          , 
                                          "safety_settings": 
                                           [
                                            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", 
                                             "threshold": "BLOCK_NONE"
                                             }
                                           , 
                                           {"category": "HARM_CATEGORY_HATE_SPEECH", 
                                           "threshold": "BLOCK_NONE"
                                           },
                                           {"category": "HARM_CATEGORY_HARASSMENT", 
                                           "threshold": "BLOCK_NONE"
                                           }, 
                                           {"category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                                            "threshold": "BLOCK_NONE"
                                            }


                                           ]
                                     }
                              }
                       )  +"\n"
                 ]

                 rf.writelines(request_list)

    temp=request_file
    client = storage.Client()
    now=datetime.strptime(str(datetime.now()),
                               '%Y-%m-%d %H:%M:%S.%f')
    # Extract name to the temp file
    temp_file = "".join([str(temp.name)])
    # Uploading the temp image file to the bucket
    dest_filename = f"{request_file_folder}/"+request_file_prefix+datetime.strftime(now, '%Y%m%d%H%M%S')+".json" 
    dest_bucket = client.get_bucket(dest_bucket_name)
    dest_blob = dest_bucket.blob(dest_filename)
    dest_blob.upload_from_filename(temp_file)

    return dest_filename

@functions_framework.http
def create_batch_request_file(request):
    """HTTP Cloud Function.
    Args:
        request (flask.Request): The request object.
        <https://flask.palletsprojects.com/en/1.1.x/api/#incoming-request-data>
    Returns:
        The response text, or any set of values that can be turned into a
        Response object using `make_response`
        <https://flask.palletsprojects.com/en/1.1.x/api/#flask.make_response>.
    """
    request_json = request.get_json(silent=True)
    request_args = request.args

    dest_bucket_name =request_args['destination_bucket']
    source_bucket_name =request_args['source_bucket']
    request_file_prefix =request_args['request_file_prefix']
    request_file_folder =request_args['request_file_folder']
    prompt_text= request_args['prompt_text']
    media_types= str(request_args['media_types']).strip().replace("[",'').\
                 replace(']','').replace("'",'').split(',')

 
    request_content= request_args['request_content']

    if request_args and 'temperature' in request_args:
        temperature= request_args['temperature']
    else:
      temperature=0.5

    if request_args and 'max_output_tokens' in request_args:
       max_output_tokens= request_args['max_output_tokens'] 
    else:
         max_output_tokens=2048

    if request_args and 'top_p' in request_args:
        top_p= request_args['top_p']
    else:
         top_p=0.5

    if request_args and 'top_k' in request_args:
        top_k= request_args['top_k']
    else:
         top_k=50
  
  
    # dest_bucket_name ='artifacts-nine-quality-test-embeddings' #request_args['destination_bucket']
    # source_bucket_name ='raw_nine_files'# request_args['source_bucket']
    # request_file_prefix = "image_request"#request_args['request_file_prefix']
    # request_file_folder =  "image_batch_request_fldr"#request_args['request_file_folder']
    # prompt_text= "You are an assistant tasked with summarizing images for retrieval. \\ These summaries will be embedded and used to retrieve the raw image. \\ Give a concise summary of the image that is well optimized for retrieval.\\ If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\\ If you are not sure about any info, please do not make it up. Do not add any extra text to the output."#request_args['prompt_text']
    # media_types= ['image/jpeg','image/png']#list(request_args['media_type'])
    # request_content= "image"#request_args['request_content']
 
    # temperature=0.5
    # max_output_tokens=2048
    # top_p=50
    # top_k=0.5

    if  request_content=='image':
      dest_filename=create_image_request_file(dest_bucket_name=dest_bucket_name,source_bucket_name=source_bucket_name,
                                      request_file_prefix=request_file_prefix,request_file_folder=request_file_folder,
                                      mime_types=media_types, prompt_text=prompt_text,temperature=temperature,
                                     max_output_tokens=max_output_tokens,top_p=top_p,top_k=top_k
                                      )  

    return {"request_file_name":dest_filename}
